# Exploration and a Simple Baseline

In this part of the assignment, we'll introduce the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) dataset, and train a Naive Bayes model as a simple baseline. The SST, introduced by [(Socher et al. 2013)](http://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf) consists of approximately 10,000 sentences from movie reviews associated with fine-grained sentiment labels on a five-point scale, and is a popular benchmark for text classification.

## Outline

- **Part (a):** The Stanford Sentiment Treebank
- **Part (b):** Naive Bayes
- **Part (c):** Exploring Negation

Exercises are interspersed throughout the notebook. Be sure you catch all of them! There are 4 questions for Part (a), 2 for Part (b), and 3 for Part (c). Some questions have multiple parts.

In [1]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("wget")      # for fetching dataset
utils.require_package("bokeh")     # for plotting histograms
utils.require_package("graphviz")  # for rendering trees

### Preliminaries: GraphViz

This notebook uses [GraphViz](https://www.graphviz.org/) to render tree structures. On Ubuntu / Debian (including Google Cloud), you can install it by running on the command line:
```
sudo apt-get install graphviz
```

For Mac OSX, you can install using Homebrew:
```
brew install graphviz
```
or see https://www.graphviz.org/download/ for more options. Run the cell below to set up rendering and show a sample tree.

In [2]:
import nltk
from w266_common import treeviz
import sst
# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

# Test rendering
print("Sample tree to test rendering:")
nltk.tree.Tree.fromstring("(4 (4 (2 I) (3 love) (3 W266)) (3 😄))")

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.
Sample tree to test rendering:


Tree('4', [Tree('4', [Tree('2', ['I']), Tree('3', ['love']), Tree('3', ['W266'])]), Tree('3', ['😄'])])

In [3]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK, NumPy, and Pandas.
import nltk
import numpy as np
import pandas as pd

# Helper libraries.
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst

# Bokeh for plotting.
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

# Helper code for plotting histograms
def plot_length_histogram(lengths, x_range=[0,100], bins=40, normed=True):
    hist, bin_edges = np.histogram(a=lengths, bins=bins, normed=normed, range=x_range)
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2
    bin_widths =  (bin_edges[1:] - bin_edges[:-1])

    hover = HoverTool(tooltips=[("bucket", "@x"), ("count", "@top")], mode="vline")
    fig = bp.figure(plot_width=800, plot_height=400, tools=[hover])
    fig.vbar(x=bin_centers, width=bin_widths, top=hist, hover_fill_color="firebrick")
    fig.y_range.start = 0
    fig.x_range.start = 0
    fig.xaxis.axis_label = "Example length (number of tokens)"
    fig.yaxis.axis_label = "Frequency"
    bp.show(fig)

Loading BokehJS ...

# Part (a): The Stanford Sentiment Treebank

_If you haven't yet, be sure to familiarize yourself with the [Prelude notebook](Prelude.ipynb), as this assignment will assume familiarity with the text pre-processing steps described there._

The [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) is one of the most widely used datasets as a benchmark for text classification. It consists of 11,855 sentences drawn from a corpus of movie reviews (originally from Rotten Tomatoes), each labeled with sentiment on a five-point scale.

For example:
```
sentence: [A warm , funny , engaging film .]
label:    4 (very positive)
```

For this assignment, we'll work with the binarized form of the dataset, where the lowest two classes are mapped to a single "negative" label, the highest two are mapped to a single "positive" label, and neutral examples are omitted.

**Side note:**
Unlike most classification datasets, SST is also a _treebank_, which means each sentence is associated with a tree structure that decomposes it into subphrases. So for the example above, we'd also have sentiment labels for `[warm , funny]` and `[engaging film .]` and so on. The trees are created by running the [Stanford Parser](https://nlp.stanford.edu/software/lex-parser.shtml) over the original sentences, then crowdsourcing sentiment labels on each sub-phrase. We'll talk more about treebanks later in the course, and in Assignment 5 you'll implement a simple version of the Stanford Parser.

For the purposes of this assignment, we'll mostly concern ourselves with the sentence-level ("root") labels, but the tree structure will come in handy in two places:
- As a way of analyzing the examples to find instances of negation
- (optionally) As a source of additional training data, by including phrase labels

### Obtaining the Data
The data is distributed as serialized trees in [S-expression](https://en.wikipedia.org/wiki/S-expression) form, like this:
```
(4 (4 (2 A) (4 (3 (3 warm) (2 ,)) (3 funny))) (3 (2 ,) (3 (4 (4 engaging) (2 film)) (2 .))))
```

We've provided an `SSTDataset` class (in `sst.py`) which will download the dataset and parse the S-expressions into [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects that you can easily view in the notebook.

`SSTDataset` also implements the text-processing pipeline described in the [Prelude notebook](Prelude.ipynb), and provides methods (`as_sparse_bow` and `as_padded_array`) to convert the data to matrix form.

Run the cell below; it will download a ~6MB .zip file to the local directory the first time you run it.

In [4]:
import sst
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")

Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


A few members of the `SSTDataset()` class that you might find useful (click through and skim read):
- **`ds.vocab`**: a [`vocabulary.Vocabulary`](https://github.com/datasci-w266/2021-spring-main/blob/master/common/vocabulary.py#L8) object managing the model vocabulary
- **`ds.{train,dev,test}_trees`**: a list of [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects representing each sentence
- **`ds.{train,dev,test}`**: a Pandas DataFrame containing the _processed_ examples, including all subphrases. `label` is the target label, `is_root` denotes whether this example is a root node (full sentence), and `root_id` is the index of the tree that the example was derived from.

In [5]:
# Look at a tree for a positive review
ds.dev_trees[3]

Tree('4', [Tree('4', [Tree('2', ['A']), Tree('4', [Tree('3', [Tree('3', ['warm']), Tree('2', [','])]), Tree('3', ['funny'])])]), Tree('3', [Tree('2', [',']), Tree('3', [Tree('4', [Tree('4', ['engaging']), Tree('2', ['film'])]), Tree('2', ['.'])])])])

In [6]:
# Look at a tree for a negative review
ds.dev_trees[361]

Tree('0', [Tree('2', ['It']), Tree('2', [Tree('2', [Tree('2', ["'s"]), Tree('0', [Tree('2', ['like']), Tree('1', [Tree('2', ['watching']), Tree('0', [Tree('1', [Tree('2', ['a']), Tree('0', ['nightmare'])]), Tree('2', [Tree('2', ['made']), Tree('2', ['flesh'])])])])])]), Tree('2', ['.'])])])

In [7]:
# Labels.
ds.train.label[0:10]

0     1
4     1
5     1
7     1
25    1
31    1
33    1
35    1
37    1
39    1
Name: label, dtype: int64

## Part (a) questions: Exploring the Data

Let's explore the data a bit, to get a sense of what we're working with. If you're not familiar with DataFrames, you may wish to review the Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/dsintro.html 

Answer the following questions in the cells below:

1. Looking at only the root examples in the training set (*Hint: use `ds.train[ds.train.is_root]`*), what is the fraction of positive labels? Is the classification task balanced, or close to it? If we used most-common-class as a baseline classifier, what would the accuracy be?
2. What are the five most common tokens (excluding punctuation) in the entire dataset? (*Hint: there are several ways to get at this - you might use `collections.Counter`, or poke around in the `ds.vocab` object.*)
3. Use the `plot_length_histogram` function (defined above) to plot a histogram of the sentence lengths in the training set. What is the 95% percentile length? (i.e. 95% of examples in the training set should be shorter than this)
4. Repeat 3., but this time including all subphrases. Notice the difference in distributions. Think about how predicting subphrases might be problematic if they're too short.

Use the cells below for your code solutions. Each part shouldn't require more than a couple lines, but you're welcome to explore more!

In [8]:
#### YOUR CODE HERE ####
# Code for Part (a).1
my_denom = len(ds.train[ds.train.is_root].label)
my_num = ds.train[ds.train.is_root].label.value_counts()[1]
print(my_num)
print(my_denom)
print(my_num/my_denom)

def balance(seq):
    from collections import Counter
    from numpy import log
    
    n = len(seq)
    classes = [(clas,float(count)) for clas,count in Counter(seq).items()]
    k = len(classes)
    
    H = -sum([ (count/n) * log((count/n)) for clas,count in classes]) #shannon entropy
    return H/log(k)

balance(ds.train[ds.train.is_root].label)

#### END(YOUR CODE) ####

3610
6920
0.5216763005780347


0.9986438398340399

In [9]:
#### YOUR CODE HERE ####
# Code for Part (a).2

my_tokens = []

for i in ds.train['tokens']:
    for j in i:
        my_tokens.append(j)
for i in ds.test['tokens']:
    for j in i:
        my_tokens.append(j)
for i in ds.dev['tokens']:
    for j in i:
        my_tokens.append(j)

c = collections.Counter(my_tokens)
c.most_common(10)

#### END(YOUR CODE) ####

[('the', 43376),
 (',', 40671),
 ('a', 34295),
 ('and', 30941),
 ('of', 28286),
 ('to', 20263),
 ('.', 19085),
 ('<unk>', 16214),
 ("'s", 14069),
 ('is', 14048)]

In [10]:
#### YOUR CODE HERE ####
# Code for Part (a).3

my_lengths = []

for i in ds.train[ds.train.is_root].tokens:
    my_lengths.append(len(i))

plot_length_histogram(my_lengths, x_range=[0,60], bins=60, normed=True)

print(np.percentile(my_lengths, 95))

#### END(YOUR CODE) ####

<ipython-input-3-ca919cac86b6>:25: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bin_edges = np.histogram(a=lengths, bins=bins, normed=normed, range=x_range)


36.0


In [11]:
#DOUBLE CHECK PERCENTILE
my_acc = []

for i in ds.train[ds.train.is_root].tokens:
    if len(i) <= 36:
        my_acc.append(1)
    else:
        my_acc.append(0)

print(len(my_acc))
print(my_acc.count(1))
print(my_acc.count(1)/len(my_acc))

6920
6627
0.9576589595375723


In [12]:
#### YOUR CODE HERE ####
# Code for Part (a).4

my_lengths = []

for i in ds.train['tokens']:
    my_lengths.append(len(i))

plot_length_histogram(my_lengths, x_range=[0,60], bins=60, normed=True)

print(np.percentile(my_lengths, 95))

#### END(YOUR CODE) ####

<ipython-input-3-ca919cac86b6>:25: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bin_edges = np.histogram(a=lengths, bins=bins, normed=normed, range=x_range)


24.0


In [13]:
#DOUBLE CHECK PERCENTILE
my_acc = []

for i in ds.train['tokens']:
    if len(i) <= 24:
        my_acc.append(1)
    else:
        my_acc.append(0)

print(len(my_acc))
print(my_acc.count(1))
print(my_acc.count(1)/len(my_acc))

98794
94188
0.9533777354900095


# Part (b): Naive Bayes

In this section, we'll build and explore a Naive Bayes model as a baseline classifier for our dataset.

Naive Bayes is perhaps the simplest possible classification algorithm, but it's one that still surprisingly effective for many text classification problems. Recall from your Machine Learning course:

$$ P(y = k) = \hat{\theta}_k = \frac{1}{N}\sum_{i = 1}^N \mathbf{1}[y_i = k] $$

$$ P(x_j | y = 1) = \hat{\theta}_{k,j} = 
\frac{ 
\sum_{i = 1}^N  \sum_{j' = 1}^{n_i} \mathbf{1}[y_i = 1 \wedge x_{j'} = j]
}{
\sum_{i = 1}^N  \mathbf{1}[y_i = 1] \cdot n_i
}
$$

where $N$ is the size of the dataset, and $n_i$ is the length (number of tokens of the $i^{th}$ example. Prediction is done by computing the score:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) $$

We'll just use the [implementation from scikit-learn](http://scikit-learn.org/stable/modules/naive_bayes.html). Like other scikit-learn classifiers, this expects the input as a `scipy.sparse` matrix. Run the cell below:

In [14]:
# 'csr' stands for "Compressed Sparse Row", which is one format
# for representing sparse matricies.
train_x_csr, train_y = ds.as_sparse_bow("train", root_only=True)
test_x_csr,  test_y  = ds.as_sparse_bow("test", root_only=True)
print("Training set: x = {:s} sparse, y = {:s}".format(str(train_x_csr.shape), 
                                                str(train_y.shape)))
print("Test set:     x = {:s} sparse, y = {:s}".format(str(test_x_csr.shape), 
                                                str(test_y.shape)))

Training set: x = (6920, 16474) sparse, y = (6920,)
Test set:     x = (1821, 16474) sparse, y = (1821,)


Note the `root_only=True` parameter - this will return only examples corresponding to whole sentences. If you set this to false, you can get examples for all phrases.

## Part (b) Questions

Answer the following questions in the **answer file found in this directory** and code cells below.

**Question 1.)** Implement Naive Bayes using `sklearn.naive_bayes.MultinomialNB`. Train on the training set and evaluate accuracy on the test set using `.predict(...)`. 

Your model should train almost instantly, and score between 82% and 83% - not bad! On SST, this is actually a very strong baseline.

Recall that Naive Bayes can be interpreted as a linear model, where score is given by:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) 
= \left( \log\hat{\theta}_1 - \log\hat{\theta}_0 \right) + \sum_{j=1}^n \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$$

You can access the values $\log\hat{\theta}_{k,j}$ from the trained model using `nb.feature_log_prob_[k,j]`.

**Question 2.)** In the cell below, compute the weights $w_j = \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$ of the linear model, and find the top 10 most negative and most positive weights. _(Hint: use `np.argsort` to get the indices of the most extreme elements.)_ Put the most negative and most positive word in the answers file.  Do the features you found make sense for this domain?

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#### YOUR CODE HERE ####
# Code for part (b).1

nb = MultinomialNB()
nb.fit(train_x_csr, train_y)
y_pred = nb.predict(test_x_csr)

#### END(YOUR CODE) ####
acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 82.21%


In [64]:
#### YOUR CODE HERE ####
# Code for part (b).2
linear_weights = []  # populate this with actual values

for j in range(0,len(nb.feature_log_prob_[0,:])):
    w_j = nb.feature_log_prob_[1,j] - nb.feature_log_prob_[0,j]
    linear_weights.append(w_j)
    #np.append(linear_weights, w_j)

top_positive_features = np.argsort(linear_weights)[-10:]
top_positive_features = top_positive_features[::-1]

top_negative_features = np.argsort(linear_weights)[:10]
    
#### END(YOUR CODE) ####

print("Most negative features:")
for idx in top_negative_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))
print("")
print("Most positive features:")
for idx in top_positive_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))

Most negative features:
  stupid (-3.17)
  suffers (-3.08)
  unfunny (-2.97)
  mess (-2.79)
  pointless (-2.79)
  flat (-2.75)
  poorly (-2.72)
  car (-2.72)
  tiresome (-2.64)
  disguise (-2.56)

Most positive features:
  powerful (3.53)
  solid (3.45)
  perfectly (2.81)
  inventive (2.69)
  refreshing (2.63)
  touching (2.56)
  riveting (2.49)
  wonderful (2.49)
  portrait (2.41)
  heartwarming (2.41)


In [51]:
print("For all features: ")
for idx in np.argsort(linear_weights):
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))

For all features: 
  stupid (-3.17)
  suffers (-3.08)
  unfunny (-2.97)
  mess (-2.79)
  pointless (-2.79)
  flat (-2.75)
  poorly (-2.72)
  car (-2.72)
  tiresome (-2.64)
  disguise (-2.56)
  seagal (-2.56)
  appears (-2.56)
  joke (-2.56)
  queen (-2.56)
  product (-2.56)
  inept (-2.48)
  lousy (-2.48)
  holes (-2.48)
  pay (-2.48)
  badly (-2.48)
  listless (-2.38)
  unnecessary (-2.38)
  nowhere (-2.38)
  irritating (-2.38)
  worst (-2.33)
  ghost (-2.28)
  plotting (-2.28)
  bother (-2.28)
  unfocused (-2.28)
  choppy (-2.28)
  logic (-2.28)
  wannabe (-2.28)
  incoherent (-2.28)
  thinks (-2.28)
  rip-off (-2.28)
  generic (-2.22)
  worse (-2.22)
  loud (-2.22)
  vapid (-2.16)
  muddled (-2.16)
  lifeless (-2.16)
  pity (-2.16)
  unless (-2.16)
  soulless (-2.16)
  exploitation (-2.16)
  crap (-2.16)
  sour (-2.16)
  tuxedo (-2.16)
  stealing (-2.16)
  unconvincing (-2.16)
  pretentious (-2.16)
  mistake (-2.16)
  awkward (-2.16)
  bad (-2.14)
  contrived (-2.12)
  bears (-2.09)

  oeuvre (-0.77)
  lower-wit (-0.77)
  mcadams (-0.77)
  fuddled (-0.77)
  ordered (-0.77)
  backlash (-0.77)
  somnambulant (-0.77)
  fire-breathing (-0.77)
  twinkly-eyed (-0.77)
  adventues (-0.77)
  snail-like (-0.77)
  aggrieved (-0.77)
  winged (-0.77)
  detachment (-0.77)
  co-wrote (-0.77)
  bidder (-0.77)
  hundreds (-0.77)
  blueprint (-0.77)
  roy (-0.77)
  subjugate (-0.77)
  webcast (-0.77)
  tear-jerking (-0.77)
  dissecting (-0.77)
  contours (-0.77)
  dispel (-0.77)
  emerged (-0.77)
  counterculture (-0.77)
  tardier (-0.77)
  noticed (-0.77)
  smeary (-0.77)
  hems (-0.77)
  delinquent (-0.77)
  hem (-0.77)
  contemptuous (-0.77)
  frankie (-0.77)
  relied (-0.77)
  woodard (-0.77)
  bazadona (-0.77)
  well-conceived (-0.77)
  novelty (-0.77)
  tick-tock (-0.77)
  yelling (-0.77)
  interlocked (-0.77)
  plumbing (-0.77)
  mistress (-0.77)
  18-year-old (-0.77)
  girlfriend (-0.77)
  22-year-old (-0.77)
  proposes (-0.77)
  christ (-0.77)
  pseudo-rock-video (-0.77)
  

  skims (-0.08)
  integrated (-0.08)
  balm (-0.08)
  stitched (-0.08)
  slovenly (-0.08)
  simpleminded (-0.08)
  essential (-0.08)
  markets (-0.08)
  dealers (-0.08)
  cake (-0.08)
  technological (-0.08)
  struck (-0.08)
  aggressive (-0.08)
  duck (-0.08)
  kingsley (-0.08)
  birds (-0.08)
  bump (-0.08)
  estranged (-0.08)
  proceeds (-0.08)
  stunningly (-0.08)
  pollution (-0.08)
  cheated (-0.08)
  102-minute (-0.08)
  vulnerable (-0.08)
  absolute (-0.08)
  bewitched (-0.08)
  illusion (-0.08)
  conception (-0.08)
  squareness (-0.08)
  old-hat (-0.08)
  passed (-0.08)
  whimper (-0.08)
  high-minded (-0.08)
  quintessentially (-0.08)
  occur (-0.08)
  rattling (-0.08)
  lights (-0.08)
  accidental (-0.08)
  accident (-0.08)
  delusions (-0.08)
  earned (-0.08)
  consideration (-0.08)
  predict (-0.08)
  puff (-0.08)
  hefty (-0.08)
  opposed (-0.08)
  durable (-0.08)
  readily (-0.08)
  punchy (-0.08)
  autobiographical (-0.08)
  accuracy (-0.08)
  overexposed (-0.08)
  real

  afghani (0.61)
  griffith (0.61)
  forster (0.61)
  meara (0.61)
  hell-bent (0.61)
  moist (0.61)
  eccentricities (0.61)
  seamy (0.61)
  commonplace (0.61)
  bewilderingly (0.61)
  sweaty-palmed (0.61)
  fairies (0.61)
  minutiae (0.61)
  betting (0.61)
  fascinates (0.61)
  over-dramatic (0.61)
  downer (0.61)
  interrogation (0.61)
  plateau (0.61)
  flourish (0.61)
  docu-drama (0.61)
  nail-biter (0.61)
  decent-enough (0.61)
  namesake (0.61)
  d.w. (0.61)
  sun-drenched (0.61)
  jiang (0.61)
  money-oriented (0.61)
  drips (0.61)
  newcastle (0.61)
  carter (0.61)
  uk (0.61)
  fetishes (0.61)
  introverted (0.61)
  disappoints (0.61)
  shortness (0.61)
  sooner (0.61)
  claustrophobia (0.61)
  hourlong (0.61)
  tolerable-to-adults (0.61)
  kid-pleasing (0.61)
  flood (0.61)
  cameras (0.61)
  arithmetic (0.61)
  formidable (0.61)
  mastering (0.61)
  fiascos (0.61)
  clouds (0.61)
  hard-won (0.61)
  chords (0.61)
  65-year-old (0.61)
  happily-ever (0.61)
  forefront (0.61

# Part (c): Examining Negation

While Naive Bayes performs well in the aggregate, as a linear model it's still limited in its ability to model complex phenomena in the data. Each feature - in this case, each word - contributes a weight to the total, and if the sum is $\ge 0$ we predict the example is positive. But what happens when we have an example with both positive and negative words? For instance:

```
[Brando 's performance fell short of the high standards set by his earlier work .]
[A thoughtful look at a painful incident that made headlines in 1995 .]
```

Run the cell below to evaluate your model on these examples. It should predict both as positive.

In [52]:
examples = ["Brando 's performance fell short of the high standards set by his earlier work .",
            "A thoughtful look at a painful incident that made headlines in 1995 ."]
canonicalized_examples = [ds.canonicalize(s.split()) for s in examples]
id_lists = [ds.vocab.words_to_ids(s) for s in canonicalized_examples]
x = utils.id_lists_to_sparse_bow(id_lists, ds.vocab.size)
nb.predict(x)

array([1, 1], dtype=int32)

## Part (c).1
<a id="answers_c1"></a>

**Question 1.)** Why does the model get the first example wrong?  Think about what other kinds of sentence structure might fail to be properly understood by a linear model.  *Hint: See the next cell.

In [67]:
linear_weights = np.asarray(linear_weights)

In [68]:
[x for x in zip(canonicalized_examples[0], linear_weights[np.array(id_lists[0])])]

[('<unk>', -0.07947581123340974),
 ("'s", 0.032134724769587564),
 ('performance', 1.0991791851082366),
 ('fell', -0.7726229917933551),
 ('short', -0.5213085635124504),
 ('of', 0.1577996227186711),
 ('the', 0.035492833505731625),
 ('high', -0.07947581123341152),
 ('standards', 0.6136713693265357),
 ('set', -0.18483632689123652),
 ('by', -0.03560912912782843),
 ('his', 0.15123398046411562),
 ('earlier', -0.995766543107564),
 ('work', 0.4801399767020129),
 ('.', 0.0184617237532958)]

In [69]:
[x for x in zip(canonicalized_examples[1], linear_weights[np.array(id_lists[1])])]

[('a', 0.1113371160989729),
 ('thoughtful', 1.9129543534567972),
 ('look', 0.829894477796171),
 ('at', 0.06570619861108806),
 ('a', 0.1113371160989729),
 ('painful', -0.19725884688979356),
 ('incident', 1.306818549886481),
 ('that', 0.01725381522514091),
 ('made', -0.16355892844395115),
 ('<unk>', -0.07947581123340974),
 ('in', 0.05422750608851956),
 ('DGDGDGDG', -0.24377886252468528),
 ('.', 0.0184617237532958)]

It's always important to look at individual examples, but let's try to do this a bit more systematically. Recall that SST gives us labels not only at the whole-sentence (root) level, but for individual phrases as well. We can use this to look for examples where polarity changes between different parts of the sentence. Here's one of the examples above:

In [70]:
ds.test_trees[210]

Tree('3', [Tree('4', [Tree('2', ['A']), Tree('4', ['thoughtful'])]), Tree('1', [Tree('1', [Tree('2', ['look']), Tree('2', [Tree('2', ['at']), Tree('2', [Tree('1', [Tree('2', ['a']), Tree('1', [Tree('1', ['painful']), Tree('2', ['incident'])])]), Tree('2', [Tree('2', ['that']), Tree('2', [Tree('3', [Tree('2', ['made']), Tree('2', ['headlines'])]), Tree('2', [Tree('2', ['in']), Tree('2', ['1995'])])])])])])]), Tree('2', ['.'])])])

The following cell will comb through the test set, looking for examples where there's some degree of ambiguity. We'll use a fairly crude heuristic for now: count up all the non-neutral phrases for a given sentence, and look at ones where there's a mix of both positive and negative labels.

In [71]:
df = ds.test

gb = df.groupby(by=['root_id'])
interesting_ids = []   # root ids, index into ds.test_trees
interesting_idxs = []  # DataFrame indices, index into ds.test
# This groups the DataFrame by sentence
for root_id, idxs in gb.groups.items():
    # Get the average score of all the phrases for this sentence
    mean = df.loc[idxs].label.mean()
    if (mean > 0.4 and mean < 0.6):
        interesting_ids.append(root_id)
        interesting_idxs.extend(idxs)
        
print("Found {:,} interesting examples".format(len(interesting_ids)))
# This will extract only the "interesting" sentences we found above
test_x_interesting, test_y_interesting = ds.as_sparse_bow("test", root_only=True, 
                                                          df_idxs=interesting_idxs)
print("Interesting ids (into ds.test_trees): ", interesting_ids)

Found 246 interesting examples
Interesting ids (into ds.test_trees):  [0, 27, 31, 32, 75, 80, 90, 96, 117, 124, 138, 140, 141, 160, 166, 186, 187, 205, 210, 212, 227, 232, 254, 269, 271, 285, 296, 307, 312, 327, 335, 373, 397, 399, 406, 407, 410, 426, 447, 511, 512, 516, 521, 534, 539, 563, 577, 588, 606, 610, 611, 637, 640, 645, 655, 662, 664, 713, 720, 721, 724, 739, 755, 758, 763, 776, 791, 793, 796, 802, 805, 810, 818, 840, 858, 887, 898, 899, 909, 910, 912, 929, 930, 961, 970, 973, 974, 975, 979, 1008, 1032, 1036, 1066, 1067, 1076, 1098, 1101, 1108, 1114, 1131, 1138, 1142, 1159, 1183, 1185, 1189, 1193, 1198, 1206, 1214, 1215, 1235, 1241, 1243, 1244, 1261, 1267, 1273, 1275, 1279, 1280, 1293, 1296, 1302, 1303, 1312, 1318, 1319, 1321, 1322, 1324, 1326, 1328, 1338, 1341, 1346, 1359, 1363, 1371, 1383, 1398, 1402, 1413, 1443, 1452, 1456, 1458, 1462, 1464, 1480, 1481, 1486, 1487, 1488, 1507, 1509, 1513, 1516, 1527, 1537, 1552, 1576, 1582, 1587, 1594, 1597, 1602, 1607, 1608, 1615, 1619, 1

## Part (c) continued

Answer the following in the cells below.

**Question 2.)** Examine a few of the "interesting" trees. What kinds of patterns do you see? What is the relation of the polarity of the sub-phrases to that of the whole sentence? Is this well-captured by a linear model?

**Question 3.)** Evaluate your model on `test_x_interesting` and compute accuracy. Does your model do better or worse on this slice of the data than on the whole test set (interesting + uninteresting examples)? What is the _relative_ change in the error rate? _(For example, if you go from 90% accuracy to 85%, that's a 50% increase in error!)_

In [80]:
#### YOUR CODE HERE ####
# Code for part (c).2
# Example: display(ds.test_trees[idx])

#ds.test_trees[0]
#ds.test_trees[1273]
#ds.test_trees[2121]
#ds.test_trees[802]
display(ds.test_trees[31])

#### END(YOUR CODE) ####

Tree('2', [Tree('2', [Tree('2', [Tree('2', ['Light']), Tree('2', [','])]), Tree('1', [Tree('3', [Tree('3', ['cute']), Tree('2', ['and'])]), Tree('0', ['forgettable'])])]), Tree('2', ['.'])])

In [85]:
#### YOUR CODE HERE ####
# Code for part (c).3

y_pred = nb.predict(test_x_csr)

acc = accuracy_score(test_y, y_pred)

#### END(YOUR CODE) ####
print("Accuracy on test set: {:.02%}".format(acc))
print("Error on test set: {:.02%}".format(1 - acc))

Accuracy on test set: 82.21%
Error on test set: 17.79%


In [86]:
#### YOUR CODE HERE ####
# Code for part (c).3

y_pred = nb.predict(test_x_interesting)
acc = accuracy_score(test_y_interesting, y_pred)

#### END(YOUR CODE) ####
print("Accuracy on selected examples: {:.02f}%".format(100*acc))
print("Error on test set: {:.02%}".format(1 - acc))

Accuracy on selected examples: 73.26%
Error on test set: 26.74%


In [87]:
my_p_change = (26.74 - 17.79)/17.79
print(my_p_change * 100)

50.30916245081506
